In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/hiv/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/hi/hiv/test_1.csv', index_col=0)

train

,smiles,value
4,O=S(=O)(O)CCS(=O)(=O)O,0
21,CC(C)CCS(=O)(=O)O,0
90,O=S(=O)(O)CCO,0
106,O=S(=O)(O)CO,0
117,O=S(=O)(O)CCCCBr,0
...,...,...
40932,COC(=O)c1cc2cc3c(c(O)c2c(=O)o1)OC1(Oc2c(O)c4c(...,0
40973,CCCCC1C(OCOc2ccccc2)COC(=O)N1C(C)c1ccccc1,0
41024,CC(C)=CC1CC(C)C2CCC(C)C3C(=O)C(O)=C(C)C(=O)C123,0
41026,CCOC(=O)C12C(=O)C(C)CCC1C(C)CC2C=C(C)C,0


# Hyperparameter Optimization

In [4]:
def run_svc_gridsearch(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
    'C': [0.1, 0.5, 1.0, 2.0, 5.0],
    'class_weight': [None, 'balanced']
    }
    svc = SVC()

    grid_search = GridSearchCV(svc, params, cv=pds, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    svc = SVC(**best_params)
    svc.fit(train_fps, train['value'])

    test_preds = svc.predict(test_fps)
    test_metrics = get_hi_metrics(test, test_preds)
    return test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

test_metrics = run_svc_gridsearch(train_maccs_fps, test_maccs_fps)
print(test_metrics)

[13:40:57] WARNING: not removing hydrogen atom without neighbors
[13:40:57] WARNING: not removing hydrogen atom without neighbors


Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV 1/1] END ..........C=0.1, class_weight=None;, score=0.150 total time=   9.1s
[CV 1/1] END ......C=0.1, class_weight=balanced;, score=0.189 total time=  31.5s
[CV 1/1] END ..........C=0.5, class_weight=None;, score=0.148 total time=   9.0s
[CV 1/1] END ......C=0.5, class_weight=balanced;, score=0.170 total time=  23.6s
[CV 1/1] END ..........C=1.0, class_weight=None;, score=0.148 total time=   9.5s
[CV 1/1] END ......C=1.0, class_weight=balanced;, score=0.157 total time=  19.6s
[CV 1/1] END ..........C=2.0, class_weight=None;, score=0.148 total time=  10.2s
[CV 1/1] END ......C=2.0, class_weight=balanced;, score=0.138 total time=  16.1s
[CV 1/1] END ..........C=5.0, class_weight=None;, score=0.181 total time=  10.5s
[CV 1/1] END ......C=5.0, class_weight=balanced;, score=0.135 total time=  13.0s
{'C': 0.1, 'class_weight': 'balanced'}
{'roc_auc': 0.6756862222709784, 'bedroc': 0.35955665399531367, 'prc_auc': 0.10727454260910

# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

    svc = SVC(
        C=5.0,
        class_weight=None
    )
    svc.fit(train_maccs_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = svc.predict(train_maccs_fps)

    test_result = test.copy()
    test_result['preds'] = svc.predict(test_maccs_fps)

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/hiv/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/hiv/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/hiv/svc_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/hiv/svc_maccs/test_{i}.csv')


[13:57:08] WARNING: not removing hydrogen atom without neighbors
[13:57:08] WARNING: not removing hydrogen atom without neighbors
[13:57:55] WARNING: not removing hydrogen atom without neighbors
[13:57:55] WARNING: not removing hydrogen atom without neighbors
[13:58:19] WARNING: not removing hydrogen atom without neighbors
[13:58:19] WARNING: not removing hydrogen atom without neighbors
